# RGI-07: Region 9 (Russian Arctic)
##### F. Maussion & S. Galos, June 2021

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import os
from utils import mkdir

## Files and storage paths

In [2]:
# Region of interest
reg = 9

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# RGI v6 file for comparison later 
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', '09_rgi60_RussianArctic.zip')

### Load the input data

In [3]:
# Read L2 files
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

### Apply selection criteria to create the RGI7 data subset

In [4]:
# For this region RGI07 shall be equal to RGI06 so...
#...extract RGI06 from GLIMS by 'subm_id' 567 which refers to data by Moholdt et al. 2012
rgi7 = shp.loc[shp['subm_id']== 567].copy()

## Write out and tar 

In [5]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
rgi7.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))

Writing...
Taring...
CompletedProcess(args=['tar', '-zcvf', '../../rgi7_data/l3_rgi7a_tar/RGI09.tar.gz', '-C', '../../rgi7_data/l3_rgi7a', 'RGI09'], returncode=0)


## New RGI-file created - Check result!

### load reference data (here RGI6) to enable comparison

In [6]:
# Just to know the name of the file to open from zip
import zipfile
with zipfile.ZipFile(rgi6_reg_file, "r") as z:
    for f in z.filelist:
        if '.shp' in f.filename:
            fname = f.filename

# load reference data
ref_odf = gpd.read_file('zip://' + rgi6_reg_file + '/' + fname)

## Compare new RGI-file and reference data set 

### Number of elements (differences do not necessarily depict major problems)

In [7]:
print('Number of glaciers in new RGI subset:', len(rgi7))
print('Number of glaciers in reference data:', len(ref_odf))
print('Difference:', len(rgi7)-len(ref_odf))

Number of glaciers in new RGI subset: 1069
Number of glaciers in reference data: 1069
Difference: 0


In [8]:
print ('number of glaciers without unique id in RGI06:', len(ref_odf)-len(ref_odf['GLIMSId'].unique()))

number of glaciers without unique id in RGI06: 0


In [9]:
print ('number of glaciers without unique id in RGI06 from GLIMS data base:', len(rgi7)-len(rgi7['glac_id'].unique()))

number of glaciers without unique id in RGI06 from GLIMS data base: 0


### How many nominal glaciers were there in RGI06-R09?

In [10]:
len(ref_odf.loc[ref_odf.Status == 2])

0

### Total area

In [11]:
# add an area field to RGI_ss and reference data
rgi7['area'] = rgi7.to_crs({'proj':'cea'}).area
ref_odf['area'] = ref_odf.to_crs({'proj':'cea'}).area

In [12]:
# print and compare area values
Area_RGI = rgi7['area'].sum() * 1e-6
print('Area RGI [km²]:', Area_RGI)
Area_ref = ref_odf['area'].sum() * 1e-6
print('Area Ref [km²]:', Area_ref)
d = (Area_RGI - Area_ref)
print('Area difference [km²]:',d)

Area RGI [km²]: 51591.52423020875
Area Ref [km²]: 51591.525007073666
Area difference [km²]: -0.0007768649156787433


### result of check:
#### the number of individual gaciers is the same! The total areas differ by only 777 m²...what else do we want?!

# End of revised noteboook